In [1]:
import numpy as np
import pandas as pd
!pip install geopandas
import geopandas as gpd
from datetime import datetime,timedelta

import bokeh.palettes as bp
from bokeh.plotting import figure,curdoc
from bokeh.transform import linear_cmap
from bokeh.layouts import column, row
from bokeh.models import (CDSView, 
						HoverTool,ColorBar,
						GeoJSONDataSource, 
						Patches,
						RadioButtonGroup,
						DateSlider,
						Button)

In [2]:
demo_url = 'https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/demographics.csv'
local_url = 'https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid_19_cases_switzerland_standard_format.csv'
case_url = 'https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid19_cases_switzerland_openzh-phase2.csv'

In [3]:
import pandas as pd
demo_raw = pd.read_csv(demo_url)
local_raw = pd.read_csv(local_url)
case_raw = pd.read_csv(case_url, low_memory=False,
                 parse_dates=["Date"])

In [4]:
demo_raw.drop(demo_raw.index[[26]])
demo_raw.head()

,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita
0,ZH,1520968,37796.0,377.96,4024.150704,0.170,258565,4472,0.002940
1,BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950
2,VD,799145,29940.0,299.40,2669.154977,0.164,131060,2268,0.002838
3,AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138
4,SG,507697,19408.0,194.08,2615.916117,0.183,92909,1565,0.003083


In [5]:
local_raw.head()

,date,country,abbreviation_canton,name_canton,lat,long,hospitalized_with_symptoms,intensive_care,total_hospitalized,home_confinment,total_currently_positive_cases,new_positive_cases,recovered,deaths,total_positive_cases,tests_performed
0,2020-03-06T00:00:00,Switzerland,AG,Aargau,47.449406,8.327495,NaN,NaN,NaN,NaN,10.0,0.0,NaN,NaN,NaN,NaN
1,2020-03-06T00:00:00,Switzerland,AI,Appenzell Innerrhoden,47.328414,9.409647,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,2020-03-06T00:00:00,Switzerland,AR,Appenzell Ausserrhoden,47.382710,9.271860,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
3,2020-03-06T00:00:00,Switzerland,BE,Bern,46.916667,7.466667,NaN,NaN,NaN,NaN,20.0,0.0,NaN,0.0,NaN,NaN
4,2020-03-06T00:00:00,Switzerland,BL,Baselland,47.482779,7.742975,NaN,NaN,NaN,NaN,9.0,0.0,NaN,NaN,NaN,NaN


In [6]:
# Extract unique 'abbreviation_canton','lat','long' combinations from local_raw
canton_point = local_raw[['abbreviation_canton','lat','long']]
canton_point.shape

(572, 3)

In [7]:
canton_point = canton_point.drop_duplicates()
canton_point.shape

(26, 3)

In [8]:
canton_point.head()

,abbreviation_canton,lat,long
0,AG,47.449406,8.327495
1,AI,47.328414,9.409647
2,AR,47.382710,9.271860
3,BE,46.916667,7.466667
4,BL,47.482779,7.742975


In [9]:
case_raw.head()

,Date,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,JU,LU,NE,NW,OW,SG,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH,CH,AG_diff,AG_pc,AG_diff_pc,AI_diff,AI_pc,AI_diff_pc,AR_diff,AR_pc,AR_diff_pc,BE_diff,BE_pc,BE_diff_pc,...,NW_diff_pc,OW_diff,OW_pc,OW_diff_pc,SG_diff,SG_pc,SG_diff_pc,SH_diff,SH_pc,SH_diff_pc,SO_diff,SO_pc,SO_diff_pc,SZ_diff,SZ_pc,SZ_diff_pc,TG_diff,TG_pc,TG_diff_pc,TI_diff,TI_pc,TI_diff_pc,UR_diff,UR_pc,UR_diff_pc,VD_diff,VD_pc,VD_diff_pc,VS_diff,VS_pc,VS_diff_pc,ZG_diff,ZG_pc,ZG_diff_pc,ZH_diff,ZH_pc,ZH_diff_pc,CH_diff,CH_pc,CH_diff_pc
0,2020-05-31,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.000000e+00,NaN
1,2020-06-01,0.0,NaN,NaN,0.0,1.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,0.000003,0.000003,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0,3.512301e-07,3.512301e-07
2,2020-06-02,0.0,NaN,0.0,1.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,2.0,20.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,1.0,9.662050e-07,9.662050e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.000011,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,4.0,0.000005,0.000005,1.0,0.000006,0.000003,0.0,0.0,0.0,2.0,0.000001,0.000001,17.0,2.341534e-06,1.990304e-06
3,2020-06-03,1.0,0.0,0.0,2.0,2.0,1.0,3.0,6.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,6.0,2.0,0.0,5.0,38.0,1.0,0.000001,0.000001,NaN,0.0,NaN,0.0,0.0,0.0,1.0,1.932410e-06,9.662050e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.000018,0.000007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,NaN,0.0,NaN,2.0,0.000008,0.000003,0.0,0.000006,0.000000,0.0,0.0,0.0,3.0,0.000003,0.000002,18.0,4.448914e-06,2.107380e-06
4,2020-06-04,3.0,NaN,0.0,4.0,2.0,1.0,5.0,8.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,NaN,9.0,4.0,0.0,13.0,62.0,2.0,0.000004,0.000003,NaN,0.0,NaN,0.0,0.0,0.0,2.0,3.864820e-06,1.932410e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000022,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000003,0.000003,NaN,0.0,NaN,3.0,0.000011,0.000004,2.0,0.000012,0.000006,0.0,0.0,0.0,8.0,0.000009,0.000005,24.0,7.258755e-06,2.809841e-06


In [10]:
print(case_raw.columns.tolist())

['Date', 'AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH', 'CH', 'AG_diff', 'AG_pc', 'AG_diff_pc', 'AI_diff', 'AI_pc', 'AI_diff_pc', 'AR_diff', 'AR_pc', 'AR_diff_pc', 'BE_diff', 'BE_pc', 'BE_diff_pc', 'BL_diff', 'BL_pc', 'BL_diff_pc', 'BS_diff', 'BS_pc', 'BS_diff_pc', 'FR_diff', 'FR_pc', 'FR_diff_pc', 'GE_diff', 'GE_pc', 'GE_diff_pc', 'GL_diff', 'GL_pc', 'GL_diff_pc', 'GR_diff', 'GR_pc', 'GR_diff_pc', 'JU_diff', 'JU_pc', 'JU_diff_pc', 'LU_diff', 'LU_pc', 'LU_diff_pc', 'NE_diff', 'NE_pc', 'NE_diff_pc', 'NW_diff', 'NW_pc', 'NW_diff_pc', 'OW_diff', 'OW_pc', 'OW_diff_pc', 'SG_diff', 'SG_pc', 'SG_diff_pc', 'SH_diff', 'SH_pc', 'SH_diff_pc', 'SO_diff', 'SO_pc', 'SO_diff_pc', 'SZ_diff', 'SZ_pc', 'SZ_diff_pc', 'TG_diff', 'TG_pc', 'TG_diff_pc', 'TI_diff', 'TI_pc', 'TI_diff_pc', 'UR_diff', 'UR_pc', 'UR_diff_pc', 'VD_diff', 'VD_pc', 'VD_diff_pc', 'VS_diff', 'VS_pc', 'VS_diff_pc', 'ZG_diff', 'Z

In [11]:
# Create a date list from case_raw and convert to datatime form
dates = case_raw['Date']
dates

0     2020-05-31
1     2020-06-01
2     2020-06-02
3     2020-06-03
4     2020-06-04
         ...    
196   2020-12-13
197   2020-12-14
198   2020-12-15
199   2020-12-16
200   2020-12-17
Name: Date, Length: 201, dtype: datetime64[ns]

In [12]:
# Read shape file from shape_dir unsing geopandas
shape_dir = r'/content/sample_data/gadm36_CHE_1.shp'
shape_raw = gpd.read_file(shape_dir)

# New Section

In [13]:
shape_raw.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,CHE,Switzerland,CHE.1_1,Aargau,Argovia|Arg¢via|Argovie,None,Canton|Kanton|Chantun,Canton,None,CH.AG,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ..."
1,CHE,Switzerland,CHE.2_1,Appenzell Ausserrhoden,Appenzell Ausser-Rhoden|Appenzell Outer Rhodes...,None,Canton|Kanton|Chantun,Canton,None,CH.AR,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ..."
2,CHE,Switzerland,CHE.3_1,Appenzell Innerrhoden,Appenzell Inner-Rhoden|Appenzell Inner Rhodes|...,None,Canton|Kanton|Chantun,Canton,None,CH.AI,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4..."
3,CHE,Switzerland,CHE.4_1,Basel-Landschaft,Bâle-Campagne|Basel-Country|Baselland|Basel-La...,None,Canton|Kanton|Chantun,Canton,None,CH.BL,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4..."
4,CHE,Switzerland,CHE.5_1,Basel-Stadt,Bâle-Ville|Basel-City|Basel-Town|Basilea-Citad...,None,Canton|Kanton|Chantun,Canton,None,CH.BS,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ..."


In [14]:
# Extract canton name abbreviations from the attribute 'HASC_1', e.g. CH.AG --> AG, CH.ZH --> ZH
# And save into a new column named 'Canton' 
shape_raw['Canton'] = shape_raw['HASC_1'].str.replace('CH.', '')
canton_poly = shape_raw[['geometry','Canton']]


In [15]:
canton_poly.head()

,geometry,Canton
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS


In [16]:
demo_raw.shape

(27, 9)

In [17]:
merged_c_d = pd.merge(canton_poly, demo_raw, on='Canton')
merged_c_d.head()

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR,55234,2231.0,22.31,2475.750784,0.197,10881,208,0.003766
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI,16145,813.0,8.13,1985.854859,0.191,3084,18,0.001115
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL,288132,9025.0,90.25,3192.598338,0.219,63101,582,0.002020
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS,194766,2628.0,26.28,7411.187215,0.199,38758,1199,0.006156


In [18]:
merged_c_d.shape

(26, 10)

In [19]:
# merged_df = pd.merge(merged_c_d, canton_point, on='Canton')
merged_df = merged_c_d.merge(canton_point.rename(columns={'abbreviation_canton':'Canton'}))
merged_df

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita,lat,long
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138,47.449406,8.327495
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR,55234,2231.0,22.31,2475.750784,0.197,10881,208,0.003766,47.382710,9.271860
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI,16145,813.0,8.13,1985.854859,0.191,3084,18,0.001115,47.328414,9.409647
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL,288132,9025.0,90.25,3192.598338,0.219,63101,582,0.002020,47.482779,7.742975
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS,194766,2628.0,26.28,7411.187215,0.199,38758,1199,0.006156,47.558395,7.573271
5,"MULTIPOLYGON (((7.09284 46.89419, 7.09202 46.8...",BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950,46.916667,7.466667
6,"MULTIPOLYGON (((6.78575 46.73603, 6.78559 46.7...",FR,318714,13998.0,139.98,2276.853836,0.157,50038,547,0.001716,46.718391,7.074008
7,"MULTIPOLYGON (((6.23440 46.33147, 6.23259 46.3...",GE,495249,9416.0,94.16,5259.653781,0.164,81221,1506,0.003041,46.195602,6.148113
8,"POLYGON ((8.90771 46.80860, 8.90603 46.81009, ...",GL,40403,1995.0,19.95,2025.213033,0.201,8121,86,0.002129,47.040570,9.068036
9,"MULTIPOLYGON (((10.22766 46.61207, 10.22734 46...",GR,198379,13863.0,138.63,1430.996177,0.213,42255,546,0.002752,46.796756,10.305946


In [29]:
daily_changes_df = pd.DataFrame() 
cases_per_canton_per_date = case_raw.iloc[: , 1:27].copy()

def calculate_daily_changes(data_toBe_treated):
  for row in range(data_toBe_treated.shape[0]):
    for col in range(data_toBe_treated.shape[1]):
      daily_changes_df.loc[row, col] = data_toBe_treated[row, col] - data_toBe_treated[row-1, col]


calculate_daily_changes(cases_per_canton_per_date.values)

daily_changes_df.columns = ['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 
                            'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH']
daily_changes_df.index=dates

# daily_changes_df.fillna(0) # Fill null with zeros as well
daily_changes_df[0:1] = 0.0 # Fill null with zeros as well for first row only
daily_changes_df.tail()

,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,JU,LU,NE,NW,OW,SG,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
Date,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-13,198.0,NaN,NaN,439.0,79.0,58.0,68.0,75.0,NaN,56.0,8.0,91.0,27.0,12.0,12.0,109.0,NaN,NaN,176.0,127.0,265.0,NaN,143.0,70.0,40.0,648.0
2020-12-14,305.0,NaN,NaN,312.0,100.0,59.0,92.0,171.0,NaN,217.0,26.0,186.0,85.0,22.0,11.0,645.0,NaN,NaN,0.0,197.0,174.0,NaN,291.0,116.0,37.0,503.0
2020-12-15,568.0,4.0,30.0,428.0,178.0,136.0,96.0,206.0,22.0,92.0,47.0,301.0,55.0,25.0,3.0,425.0,50.0,142.0,55.0,241.0,213.0,19.0,284.0,75.0,57.0,1003.0
2020-12-16,535.0,6.0,37.0,679.0,170.0,98.0,88.0,111.0,15.0,116.0,39.0,304.0,48.0,25.0,12.0,241.0,57.0,218.0,106.0,114.0,318.0,24.0,230.0,16.0,62.0,938.0
2020-12-17,NaN,8.0,51.0,613.0,150.0,119.0,85.0,NaN,56.0,NaN,NaN,NaN,NaN,27.0,14.0,NaN,40.0,178.0,108.0,0.0,258.0,12.0,NaN,98.0,63.0,1108.0


In [24]:
merged_df.shape, dates.shape, daily_changes_df.shape

((26, 12), (201,), (201, 26))

In [30]:
# For each date, extract a list of daily new cases per capita from all cantons(e.g. 'AG_diff_pc', 'AI_diff_pc', etc.), and add as a new column in merged
# For instance, in the column['2020-10-31'], there are: [0.0005411327220155498, nan, nan, 0.000496300306803826, ...]

for i,d in enumerate(dates):
  merged_df[d] = daily_changes_df.iloc[i].values

merged_df.tail()

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita,lat,long,2020-05-31 00:00:00,2020-06-01 00:00:00,2020-06-02 00:00:00,2020-06-03 00:00:00,2020-06-04 00:00:00,2020-06-05 00:00:00,2020-06-06 00:00:00,2020-06-07 00:00:00,2020-06-08 00:00:00,2020-06-09 00:00:00,2020-06-10 00:00:00,2020-06-11 00:00:00,2020-06-12 00:00:00,2020-06-13 00:00:00,2020-06-14 00:00:00,2020-06-15 00:00:00,2020-06-16 00:00:00,2020-06-17 00:00:00,2020-06-18 00:00:00,2020-06-19 00:00:00,2020-06-20 00:00:00,2020-06-21 00:00:00,2020-06-22 00:00:00,2020-06-23 00:00:00,2020-06-24 00:00:00,2020-06-25 00:00:00,2020-06-26 00:00:00,2020-06-27 00:00:00,...,2020-11-08 00:00:00,2020-11-09 00:00:00,2020-11-10 00:00:00,2020-11-11 00:00:00,2020-11-12 00:00:00,2020-11-13 00:00:00,2020-11-14 00:00:00,2020-11-15 00:00:00,2020-11-16 00:00:00,2020-11-17 00:00:00,2020-11-18 00:00:00,2020-11-19 00:00:00,2020-11-20 00:00:00,2020-11-21 00:00:00,2020-11-22 00:00:00,2020-11-23 00:00:00,2020-11-24 00:00:00,2020-11-25 00:00:00,2020-11-26 00:00:00,2020-11-27 00:00:00,2020-11-28 00:00:00,2020-11-29 00:00:00,2020-11-30 00:00:00,2020-12-01 00:00:00,2020-12-02 00:00:00,2020-12-03 00:00:00,2020-12-04 00:00:00,2020-12-05 00:00:00,2020-12-06 00:00:00,2020-12-07 00:00:00,2020-12-08 00:00:00,2020-12-09 00:00:00,2020-12-10 00:00:00,2020-12-11 00:00:00,2020-12-12 00:00:00,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00
21,"POLYGON ((8.48248 46.52972, 8.48077 46.53087, ...",UR,36433,2000.0,20.00,1821.650000,0.204,7432,63,0.001729,46.880422,8.644409,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.0,15.0,14.0,15.0,NaN,NaN,13.0,31.0,24.0,5.0,39.0,NaN,NaN,NaN,20.0,10.0,10.0,19.0,NaN,NaN,NaN,22.0,0.0,14.0,19.0,NaN,NaN,NaN,16.0,13.0,10.0,16.0,25.0,NaN,NaN,19.0,24.0,12.0
22,"POLYGON ((7.61459 45.97337, 7.61351 45.97326, ...",VS,343955,18463.0,184.63,1862.942100,0.196,67415,834,0.002425,46.209567,7.604659,0.0,0.0,4.0,2.0,3.0,2.0,2.0,2.0,1.0,3.0,1.0,4.0,7.0,0.0,1.0,3.0,8.0,5.0,2.0,6.0,2.0,2.0,11.0,6.0,9.0,3.0,6.0,2.0,...,743.0,1354.0,1299.0,1009.0,946.0,859.0,636.0,376.0,797.0,665.0,564.0,536.0,455.0,368.0,252.0,530.0,360.0,420.0,393.0,318.0,214.0,190.0,353.0,308.0,370.0,361.0,347.0,251.0,165.0,368.0,288.0,325.0,280.0,246.0,196.0,143.0,291.0,284.0,230.0,NaN
23,"MULTIPOLYGON (((7.05791 46.86422, 7.05659 46.8...",VD,799145,29940.0,299.40,2669.154977,0.164,131060,2268,0.002838,46.536900,6.584780,0.0,1.0,1.0,0.0,2.0,2.0,1.0,2.0,5.0,1.0,4.0,4.0,4.0,0.0,0.0,5.0,4.0,1.0,1.0,8.0,4.0,2.0,4.0,6.0,10.0,8.0,8.0,4.0,...,191.0,554.0,426.0,343.0,318.0,303.0,180.0,119.0,323.0,282.0,213.0,195.0,152.0,119.0,94.0,220.0,165.0,145.0,143.0,156.0,77.0,74.0,163.0,126.0,120.0,126.0,137.0,62.0,63.0,177.0,78.0,142.0,116.0,122.0,72.0,70.0,116.0,75.0,16.0,98.0
24,"POLYGON ((8.66821 47.16368, 8.67379 47.16357, ...",ZG,126837,3306.0,33.06,3836.569873,0.170,21562,206,0.001624,47.157296,8.537294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,4.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,2.0,0.0,...,55.0,38.0,37.0,56.0,56.0,81.0,55.0,47.0,36.0,57.0,65.0,69.0,48.0,60.0,39.0,43.0,43.0,58.0,60.0,41.0,62.0,40.0,27.0,44.0,56.0,69.0,46.0,70.0,32.0,33.0,41.0,52.0,52.0,47.0,63.0,40.0,37.0,57.0,62.0,63.0
25,"POLYGON ((8.87364 47.24979, 8.86491 47.24844, ...",ZH,1520968,37796.0,377.96,4024.150704,0.170,258565,4472,0.002940,47.451542,8.564572,0.0,0.0,2.0,3.0,8.0,6.0,4.0,0.0,0.0,0.0,3.0,2.0,2.0,3.0,5.0,7.0,7.0,10.0,5.0,1.0,7.0,5.0,3.0,4.0,16.0,8.0,19.0,12.0,...,862.0,472.0,715.0,1012.0,813.0,780.0,719.0,603.0,370.0,864.0,965.0,782.0,724.0,727.0,352.0,324.0,794.0,812.0,666.0,762.0,729.0,420.0,374.0,780.0,942.0,770.0,973.0,795.0,458.0,506.0,1012.0,1056.0,938.0,1039.0,766.0,648.0,503.0,1003.0,938.0,1108.0
